In [1]:
import vk_api
import random
import time

In [9]:
posts = []
for offset in range(100, 1001, 100):
    vk = vk_api.VkApi(token="686ffbfe686ffbfe686ffbfe0c680303e56686f686ffbfe35ec2f6c5653f9b767d8ae65")
    vk_session = vk.get_api()
    poroshok = vk_session.wall.get(owner_id = -31481258, offset = offset, count = 100)
    posts.extend(poroshok['items'])
    print(f"Finished first {offset} posts")
    time.sleep(random.uniform(2, 10))

Finished first 100 posts
Finished first 200 posts
Finished first 300 posts
Finished first 400 posts
Finished first 500 posts
Finished first 600 posts
Finished first 700 posts
Finished first 800 posts
Finished first 900 posts
Finished first 1000 posts


In [13]:
posts[0]

{'inner_type': 'wall_wallpost',
 'comments': {'can_post': 0, 'count': 0},
 'marked_as_ads': 0,
 'hash': 'ndp9jmCoA5ww2IaX8A',
 'type': 'post',
 'attachments': [],
 'date': 1689249600,
 'from_id': -31481258,
 'id': 107058,
 'likes': {'can_like': 1,
  'count': 112,
  'user_likes': 0,
  'can_publish': 1,
  'repost_disabled': False},
 'owner_id': -31481258,
 'post_source': {'type': 'vk'},
 'post_type': 'post',
 'reposts': {'count': 12, 'user_reposted': 0},
 'text': 'критяне смотрятся отсюда\nкак насекомые на мхе\nспасибо папа я прям рею\nпрям хе\n© [id33742928|Нестер Пим]',
 'views': {'count': 8425}}